<a href="https://colab.research.google.com/github/VIJAYRUR/DL/blob/main/Copy_of_PS2_VGG_dogs_and_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow,keras

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
# from keras.applications.imagenet_utils import _obtain_input_shape # this will work for older versions of keras. 2.2.0 or before
# from keras.engine.topology import get_source_inputs

In [ ]:
# model building

def VGGupdated(input_tensor=None,classes=2):

    img_rows, img_cols = 300, 300   # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)
    if input_tensor is None:
        img_input = Input(shape=img_dim)
    else:
        img_input = Input(tensor=input_tensor, shape=img_dim)



    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)


    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.


    model = Model(inputs = img_input, outputs = x, name='VGGdemo')


    return model

In [ ]:
model = VGGupdated(classes = 2) # bedroom and diningroom

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/test')

animal_types = os.listdir('/content/drive/MyDrive/test')
print (animal_types)  #what kinds of animals are in this dataset

print("Types of rooms found: ", len(dataset_path))

In [ ]:
lst = []

for item in animal_types:
    # Get all the file names
    all_animals = os.listdir('/content/drive/MyDrive/test' + '/' +item)
    # Add them to the lst
    for i in all_animals:
        lst.append((item, str('/content/drive/MyDrive/test' + '/' +item) + '/' + i))
print(lst)

In [ ]:
# Build a dataframe
animals_df = pd.DataFrame(data=lst, columns=['animal name', 'image'])
print(animals_df.head())

In [ ]:
# Let's check how many samples for each category are present
print("Total number of rooms in the dataset: ", len(animals_df))

animals_count = animals_df['animal name'].value_counts()

print("animals in each category: ")
print(animals_count)

In [ ]:
import cv2
path = '/content/drive/MyDrive/test/'


im_size = 300

images = []
labels = []
#resizing and adding images to img list and labels to labels list
for i in animal_types:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)


In [ ]:
images = np.array(images)

images = images.astype('float32') / 255*1.0
images.shape


In [ ]:
#pre processing data
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=animals_df['animal name'].values
print(y[:5])

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

In [ ]:
#encoding labels
y=y.reshape(-1,1)
onehotencoder = OneHotEncoder()  #Converted  scalar output into vector output where the correct class will be 1 and other will be 0
Y= onehotencoder.fit_transform(y)
Y.shape

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.3, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
from scipy.sparse import csr_matrix

train_y = csr_matrix(train_y) # convert to SparseTensor if not already done
train_y = np.array(train_y.toarray()) # convert to dense numpy array


In [ ]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=10, batch_size=32)


In [ ]:
from scipy.sparse import csr_matrix

test_y = csr_matrix(test_y) # convert to SparseTensor if not already done
test_y = np.array(test_y.toarray()) # convert to dense numpy array

preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("accuracy = " + str(preds[1]))

In [ ]:
from matplotlib.pyplot import imread, imshow
import tensorflow as tf
img_path = '/content/drive/MyDrive/Classroom/dog.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(300, 300))

x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

print('Input image shape:', x.shape)

my_image = imread(img_path)
imshow(my_image)


In [ ]:
print(model.predict(x))

In [ ]:
# hence dog agaya but model accuracy is 50 percent , feed more data